In [1]:
import pandas as pd

#import numpy as np
#from numpy import genfromtxt, savetxt

#import random
#from random import sample

#import matplotlib.pyplot as plt

#from sklearn.decomposition import PCA

import xgboost as xgb

#from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, cross_val_score

#from sklearn.grid_search import GridSearchCV

#from sklearn.metrics import accuracy_score

import csv
from numpy import arange

In [2]:
def read_numtrain(path):
    X_all_train=pd.read_csv(path, usecols=list(range(969)))
    response=pd.read_csv(path, usecols=[969])
    
    return X_all_train, response
    
    
    

In [3]:
def mcc_scorer(predd,yd):
    #pred=predd.get_label()
    pred=predd.tolist()
    print("max: %.15f  min:%.15f"%(min(pred),max(pred)))
    y=yd.get_label()
    can=list(arange(min(pred)+0.00001,max(pred),0.015))
    #can=list(arange(0.1,0.5,0.015))
    tp=[sum([1 for i in range(len(pred)) if pred[i]>=c and round(y[i])==1]) for c in can]
    tn=[sum([1 for i in range(len(pred)) if pred[i]<c and round(y[i])==0]) for c in can]
    fp=[sum([1 for i in range(len(pred)) if pred[i]>=c and round(y[i])==0]) for c in can]
    fn=[sum([1 for i in range(len(pred)) if pred[i]<c and round(y[i])==1]) for c in can]
    #print('TP:%d,TN:%d,FP:%d,FN:%d'%(tp,tn,fp,fn))
    score=[(tp[i]*tn[i]-fp[i]*fn[i])/(((tp[i]+fp[i])*(tp[i]+fn[i])*(tn[i]+fp[i])*(tn[i]+fn[i]))**(1/2.0))
           for i in range(len(can))]
    fmax=max(score)
    p=score.index(fmax)
    threshold=can[p]
    print('threshold:',threshold)
    print('TP:%d,TN:%d,FP:%d,FN:%d'%(tp[p],tn[p],fp[p],fn[p]))
    return('mcc',fmax)

In [4]:
def num_feature(X_train,response):
    column_name=X_train.columns
    
    grid = {'max_depth':8,
        #'booster': 'gbtree',
        'objective': 'binary:logistic',
        'subsample': 1,
        'colsample_bytree':0.95,
        'learning_rate': 0.1,
        'max_depth': 8,
        'seed': 2016,
        'silent': 0,
        'gamma':0.05}

    clf1 = xgb.XGBClassifier()
    clf1.set_params(**grid)

    clf1.fit(X_train,response['Response'])
    
    importance=clf1.feature_importances_
    
    importance_s=sorted([(index,value) for index, value in enumerate(importance.tolist())],key=lambda x: x[1],reverse = True)
    
    important_feature_index=[index for (index,val) in importance_s if val>0.003]
    
    col_name = [column_name[index] for index in important_feature_index]
    
    return col_name
    

In [5]:
def main():
    path='Y://Data//Kaggle//bosch//train_numeric//train_numeric.csv'
    X_train,response=read_numtrain(path)
    cols=num_feature(X_train,response)
    print(cols)
    

In [ ]:
main()

In [6]:
#10.25 main()

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['Id', 'L3_S38_F3952', 'L3_S29_F3407', 'L3_S30_F3759', 'L3_S33_F3865', 'L3_S30_F3744', 'L0_S0_F20', 'L3_S30_F3754', 'L3_S29_F3351', 'L3_S33_F3859', 'L3_S36_F3920', 'L3_S32_F3850', 'L1_S24_F1723', 'L3_S38_F3960', 'L3_S29_F3479', 'L3_S30_F3804', 'L3_S33_F3857', 'L3_S30_F3809', 'L3_S30_F3704', 'L3_S29_F3339', 'L3_S30_F3829', 'L1_S24_F1846', 'L3_S29_F3373', 'L3_S29_F3348', 'L3_S30_F3769', 'L3_S29_F3345', 'L2_S26_F3040', 'L3_S30_F3494', 'L0_S1_F24', 'L3_S30_F3554', 'L2_S26_F3106', 'L3_S38_F3956', 'L0_S1_F28', 'L3_S29_F3324', 'L3_S29_F3333', 'L3_S29_F3342', 'L3_S30_F3774', 'L3_S30_F3574', 'L0_S0_F16', 'L3_S35_F3889', 'L1_S24_F1844', 'L3_S29_F3327', 'L3_S29_F3336', 'L3_S29_F3321', 'L3_S30_F3609', 'L3_S30_F3794', 'L3_S35_F3896', 'L3_S30_F3629', 'L3_S30_F3749', 'L0_S12_F350', 'L3_S29_F3354', 'L3_S29_F3379', 'L1_S24_F1632', 'L3_S29_F3427', 'L0_S0_F18', 'L1_S24_F1498', 'L3_S29_F3330', 'L3_S30_F3604', 'L1_S24_F1514', 'L1_S24_F1575', 'L3_S29_F3382', 'L3_S29_F3473', 'L3_S30_F3534', 'L1_S24_F867', 'L

In [26]:
importance_s

[(883, 0.013615733943879604),
 (854, 0.01242705900222063),
 (826, 0.011886751279234886),
 (751, 0.011778689920902252),
 (829, 0.011670628562569618),
 (828, 0.011346444487571716),
 (851, 0.01037389226257801),
 (850, 0.010157769545912743),
 (360, 0.009941646829247475),
 (839, 0.009725524112582207),
 (872, 0.009725524112582207),
 (838, 0.009617462754249573),
 (10, 0.009509401395916939),
 (848, 0.009509401395916939),
 (735, 0.009077155962586403),
 (885, 0.009077155962586403),
 (771, 0.008212665095925331),
 (831, 0.008212665095925331),
 (799, 0.008104603737592697),
 (394, 0.007780419196933508),
 (682, 0.007780419196933508),
 (731, 0.00756429648026824),
 (741, 0.00756429648026824),
 (776, 0.00756429648026824),
 (843, 0.00756429648026824),
 (818, 0.007240112405270338),
 (827, 0.007132051046937704),
 (832, 0.006807866971939802),
 (393, 0.006699805613607168),
 (726, 0.006591744255274534),
 (734, 0.006591744255274534),
 (743, 0.0064836828969419),
 (788, 0.0064836828969419),
 (13, 0.0063756215386